In [1]:
import os
import pandas as pd
import re
from bs4 import BeautifulSoup
from urllib import request
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import typing
import time

In [26]:
os.listdir("./data/BLAST/")

['NC_003861.html',
 'OK362288.html',
 'OK377341.html',
 'OK377342.html',
 'OK377343.html',
 'OK424595.html']

In [21]:
contents = {}

for file in os.listdir("./data/BLAST/"):
    with open("./data/BLAST/" + file, 'r') as fp:
        contents[file.replace(".html", "")] = BeautifulSoup(fp.read(), "html.parser")

In [33]:
contents["NC_003861"].find("table", attrs = {"id": "dscTable"});

In [34]:
# a typical table record

"""

<tr id="dtr_61657622" ind="7" class="odd dflLnk">
<td class="l c0"><span class="ind">7</span><input type="checkbox" id="chk_7" class="cb" name="getSeqGi" value="61657622" onclick="configDescrLinks(event,this)"><label for="chk_7" onclick="checkShiftKey(event,this)"><span class="accsb">Select seq AJ890224.1</span></label></td>
# Need this!
<td class="ellipsis c2"><span><a title="Go to alignment for Sri Lankan cassava mosaic virus ac1 gene, ac2 gene, ac3 gene, ac4 gene, av1 gene and av2 gene, isolate Kerala 15, complete virus segment" class="deflnDesc" hsp="1" len="2761" ind="7" accs="AJ890224.1" seqfsta="gi|61657622" gi="61657622" seqid="61657622" id="deflnDesc_7" onclick="DisplayAlignFromDescription(this);" href="#alnHdr_61657622">Sri Lankan cassava mosaic virus ac1 gene, ac2 gene, ac3 gene, ac4 gene, av1 gene and av2 gene, isolate Kerala 15, complete virus segment</a></span></td>
<td class="ellipsis c3"><span><a href="https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=161378" class="sciName" title="Taxonomy for Sri Lankan cassava mosaic virus" target="lnk<@rid@>">Sri Lankan cassava mosaic virus</a></span></td>
<td class="ellipsis c4 ui-ncbigrid-column-hidden"><span><a href="https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=161378" class="cmnName" title="Taxonomy for NA">NA</a></span></td>
<td class="c5 ui-ncbigrid-column-hidden"><a href="https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=161378" class="txid" title="Taxonomy for taxid 161378">161378</a></td>
<td class="c6">4102</td>
<td class="c7">4102</td>
<td class="c8">100%</td>
<td class="c9">0.0</td>
# Need this! (Percentage identity)
<td class="c10">93.55%</td>
# Need this!
<td class="c11 acclen">2756</td>
<td class="c12 l lim">
# Need this! (Link and the accession number)
<a href="https://www.ncbi.nlm.nih.gov/nucleotide/AJ890224.1?report=genbank&amp;log$=nucltop&amp;blast_rank=7&amp;RID=H5G050J9016" class="dflSeq" target="lnkH5G050J9016" title="Show report for AJ890224.1">AJ890224.1</a>
</td>
</tr>

"""

'\n\n<tr id="dtr_61657622" ind="7" class="odd dflLnk">\n<td class="l c0"><span class="ind">7</span><input type="checkbox" id="chk_7" class="cb" name="getSeqGi" value="61657622" onclick="configDescrLinks(event,this)"><label for="chk_7" onclick="checkShiftKey(event,this)"><span class="accsb">Select seq AJ890224.1</span></label></td>\n<td class="ellipsis c2"><span><a title="Go to alignment for Sri Lankan cassava mosaic virus ac1 gene, ac2 gene, ac3 gene, ac4 gene, av1 gene and av2 gene, isolate Kerala 15, complete virus segment" class="deflnDesc" hsp="1" len="2761" ind="7" accs="AJ890224.1" seqfsta="gi|61657622" gi="61657622" seqid="61657622" id="deflnDesc_7" onclick="DisplayAlignFromDescription(this);" href="#alnHdr_61657622">Sri Lankan cassava mosaic virus ac1 gene, ac2 gene, ac3 gene, ac4 gene, av1 gene and av2 gene, isolate Kerala 15, complete virus segment</a></span></td>\n<td class="ellipsis c3"><span><a href="https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=161378" class

In [48]:
res_table = contents["NC_003861"].find("table", attrs = {"id": "dscTable"})
trows = res_table.find_all("tr", attrs = {"class": ["odd dflLnk", "even dflLnk"]})

trows[0]

<tr class="odd dflLnk" id="dtr_20806048" ind="1">
<td class="l c0"><span class="ind">1</span><input class="cb" id="chk_1" name="getSeqGi" onclick="configDescrLinks(event,this)" type="checkbox" value="20806048"/><label for="chk_1" onclick="checkShiftKey(event,this)"><span class="accsb">Select seq NC_003861.1</span></label></td>
<td class="ellipsis c2"><span><a accs="NC_003861.1" class="deflnDesc" gi="20806048" href="#alnHdr_20806048" hsp="1" id="deflnDesc_1" ind="1" len="2755" onclick="DisplayAlignFromDescription(this);" seqfsta="gi|20806048" seqid="20806048" stat="disp" title="Go to alignment for Sri Lankan cassava mosaic virus-[Colombo] DNA-A, complete genome, isolate SLCMV-Col &gt;emb|AJ314737.1| Sri Lankan cassava mosaic virus-[Colombo] DNA-A, complete genome, isolate SLCMV-Col">Sri Lankan cassava mosaic virus-[Colombo] DNA-A, complete genome, isolate SLCMV-Col</a></span></td>
<td class="ellipsis c3"><span><a class="sciName" href="https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax

In [57]:
# Design the logic to extract the needed details from each table row.

# Extract the name
trows[0].find("a", attrs = {"class": "deflnDesc"}).text

# Extract the percentage identity and sequence length
trows[0].find_all("td", attrs = {"class": ["c10", "c11 acclen"]})

# Extract the accession id 
trows[0].find("a", attrs = {"class": "dflSeq"}).text

# Extract the genbank link
trows[0].find("a", attrs = {"class": "dflSeq"}).get("href")

'https://www.ncbi.nlm.nih.gov/nucleotide/NC_003861.1?report=genbank&log$=nucltop&blast_rank=1&RID=H5G050J9016'

In [71]:
def parse_blast_report(file_path: str) -> dict[str: list[str]]:

    """
    
    """
    
    assert (file_path.endswith(".html") or file_path.endswith(".htm")), "Only HTML files are supported!"
    with open(file_path, 'r') as fp:
        soup = BeautifulSoup(fp.read(), "html.parser")

    table = soup.find("table", attrs = {"id": "dscTable"})
    assert table, "There isn't a <table> tag with id=\"dscTable\" in the HTML file!"

    trows = res_table.find_all("tr", attrs = {"class": ["odd dflLnk", "even dflLnk"]})
    assert trows, "There isn't a <tr> tag with class=\"odd dflLnk\" or class=\"even dflLnk\" in the HTML file!"

    result = {
        "Name": [],
        "PercntIdentity": [],
        "SeqLength": [],
        "Accession": [],
        "GenBankLink": []
    }

    for trow in trows:
        # Extract the name
        result["Name"].append(trow.find("a", attrs = {"class": "deflnDesc"}).text)

        # Extract the percentage identity and sequence length
        prcntident_and_len = trow.find_all("td", attrs = {"class": ["c10", "c11 acclen"]})
        result["PercntIdentity"].append(prcntident_and_len[0].text)
        result["SeqLength"].append(prcntident_and_len[1].text)
        
        # Extract the accession id 
        result["Accession"].append(trow.find("a", attrs = {"class": "dflSeq"}).text)
        
        # Extract the genbank link
        result["GenBankLink"].append(trow.find("a", attrs = {"class": "dflSeq"}).get("href"))

    return result

In [83]:
df = pd.DataFrame()

root = "./data/BLAST/"

for file in os.listdir(root):
    temp = pd.DataFrame(parse_blast_report(root + file))
    df = pd.concat([df, temp], axis = 0)

In [85]:
df.shape

(600, 5)

In [88]:
# Well, if that isn't disappointing

df.drop_duplicates().shape

(100, 5)

In [90]:
df.to_csv("./data/BLAST/results.csv", sep = ',', index = None)

In [3]:
# BLAST results

blast = pd.read_csv("./data/BLAST/results.csv")
blast.shape

(100, 6)

In [3]:
# FASTA query format => https://www.ncbi.nlm.nih.gov/nuccore/NC_003861.1?report=fasta

blast.Accession

0     NC_003861.1
1      KP455484.1
2      OM857615.1
3      AJ890226.1
4      KU550961.1
         ...     
95     MW816856.1
96     JN135236.3
97     MW816855.1
98    NC_038956.1
99     OK236813.1
Name: Accession, Length: 100, dtype: object

In [35]:
os.listdir("./data/BLAST/")

['blastseqs',
 'NC_003861.html',
 'OK362288.html',
 'OK377341.html',
 'OK377342.html',
 'OK377343.html',
 'OK424595.html',
 'results.csv']

In [21]:
URL_TEMPLATE: str = "https://www.ncbi.nlm.nih.gov/nuccore/{}?report=fasta"

In [6]:
# Yes, Edge!

service = webdriver.EdgeService()
options = webdriver.EdgeOptions()
options.add_argument("--headless")

wdriver = webdriver.Edge(options, service)

In [4]:
WRITE_PATH_TEMPLATE = "./data/BLAST/blastseqs/{}.fasta"
URL_TEMPLATE = "https://www.ncbi.nlm.nih.gov/nuccore/{}?report=fasta"

for accession in blast.Accession:
    
    wdriver.get(URL_TEMPLATE.format(accession))
    
    # Wait until the <div id="viewercontent1" class="seq gbff" ....> tag is loaded.
    # 10 seconds wait
    time.sleep(5)
    # _ = WebDriverWait(wdriver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "seq gbff")))
    
    response = wdriver.page_source
    assert response     # Make sure it's not empty.

    soup = BeautifulSoup(response, "html.parser")
    div = soup.find("div", attrs = {"class": "seq gbff", "id": "viewercontent1"})
    
    fasta = div.find("pre").text

    # remove the trailing digits after the period in file names.
    with open(WRITE_PATH_TEMPLATE.format(accession.split('.')[0]), 'w') as fp:
        fp.write(fasta)

wdriver.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=117.0.2045.43)
Stacktrace:
	GetHandleVerifier [0x00007FF62C5299A2+63682]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF62C4B18F2+267714]
	(No symbol) [0x00007FF62C26AC0E]
	(No symbol) [0x00007FF62C2428C7]
	(No symbol) [0x00007FF62C2E0B9F]
	(No symbol) [0x00007FF62C2F63E8]
	(No symbol) [0x00007FF62C2DC623]
	(No symbol) [0x00007FF62C2ADA7A]
	(No symbol) [0x00007FF62C2ACD6B]
	(No symbol) [0x00007FF62C2AE204]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF62C6F6779+1233737]
	(No symbol) [0x00007FF62C32B274]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF62C3F329A+33498]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF62C3EBDD9+3609]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF62C6F5514+1229028]
	Microsoft::Applications::

In [30]:
blast.Accession[1]

'KP455484.1'

In [6]:
service = webdriver.EdgeService()
options = webdriver.EdgeOptions()
wdriver = webdriver.Edge(options, service)

meta_info = []

TEMPLATE_URL = "https://www.ncbi.nlm.nih.gov/nuccore/{}"

for i, accession in enumerate(blast.Accession):
    
    wdriver.get(TEMPLATE_URL.format(accession))
    time.sleep(5)

    soup = BeautifulSoup(wdriver.page_source, "html.parser")
    location = soup.find("span", attrs = {"class": "feature", "id": f"feature_{accession}_source_0"})
    # DO NOT CALL THE .text METHOD HERE! IF IT'S NONE, ERROR WILL NOT BE HANDLED!
    
    if location:
        # INVOKING THE .text HERE, INTENTIONALLY!
        meta_info.append(location.text.replace('\n', '').replace(' ', '').split('/'))
        print(f"{i:2} {accession} Success!")
    else:
        print(f"{1:2} {accession} Failed")
    
    
wdriver.quit()    

 0 NC_003861.1 Success!
 1 KP455484.1 Success!
 2 OM857615.1 Success!
 3 AJ890226.1 Success!
 4 KU550961.1 Success!
 5 AJ890225.1 Success!
 6 AJ890224.1 Success!
 7 KR611577.1 Success!
 8 OM715155.1 Success!
 9 OM715161.1 Success!
10 OK571385.1 Success!
11 AJ579307.1 Success!
12 AJ607394.1 Success!
13 KT861468.1 Success!
14 OM715160.1 Success!
15 MZ605403.1 Success!
16 MZ605401.1 Success!
17 MT671423.1 Success!
18 MT671419.1 Success!
19 MT671417.1 Success!
20 MN026159.1 Success!
21 LC382268.1 Success!
22 MW854732.1 Success!
23 LC485944.1 Success!
24 LC485942.1 Success!
25 KC424490.1 Success!
26 MT671415.1 Success!
27 OM857621.1 Success!
28 LC586845.1 Success!
29 MT017511.1 Success!
30 MN544647.1 Success!
31 ON455005.1 Success!
32 MT671421.1 Success!
33 MN577577.1 Success!
34 MN577575.1 Success!
35 MN026160.1 Success!
36 LC312131.1 Success!
37 OQ745820.1 Success!
38 LC382266.1 Success!
39 OM715156.1 Success!
40 KF898349.1 Success!
41 MT671413.1 Success!
42 MN688216.1 Success!
43 MT67140

In [16]:
with open("./data/BLAST/metainfo.csv", mode = 'w') as fp:
    for i, info in enumerate(meta_info, start = 1):
        fp.writelines(f"{i}, " + ", ".join(info) + '\n')

In [12]:
", ".join(meta_info[0])

'source1..2755, organism="SriLankancassavamosaicvirus-[Colombo]", mol_type="genomicDNA", isolate="SLCMV-Col", db_xref="taxon:223328", segment="DNAA", clone="pSL1", lab_host="Nicotianabenthamiana", country="SriLanka:Columbo"'

In [6]:
blast = pd.read_csv("./data/BLAST/metainfo.csv", header = None)

In [7]:
blast.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1,source1..2755,"organism=""SriLankancassavamosaicvirus-[Colombo]""","mol_type=""genomicDNA""","isolate=""SLCMV-Col""","db_xref=""taxon:223328""","segment=""DNAA""","clone=""pSL1""","lab_host=""Nicotianabenthamiana""","country=""SriLanka:Columbo""",NaN,NaN,NaN,NaN,NaN,NaN
1,2,source1..2758,"organism=""SriLankancassavamosaicvirus""","mol_type=""genomicDNA""","isolate=""Attur2""","host=""cassava;cultivar:H226""","db_xref=""taxon:161378""","segment=""DNAA""","country=""India""","collection_date=""19-Jan-2009""",NaN,NaN,NaN,NaN,NaN,NaN
2,3,source1..2757,"organism=""SriLankancassavamosaicvirus""","mol_type=""genomicDNA""","isolate=""DSMZPV-0424""","host=""Manihotesculenta""","db_xref=""taxon:161378""","segment=""DNA-A""","lab_host=""Nicotianabenthamiana""","country=""India:Kerala""",NaN,NaN,NaN,NaN,NaN,NaN
3,4,source1..2757,"organism=""SriLankancassavamosaicvirus-India[I...","mol_type=""genomicDNA""","isolate=""KeralaC4""","isolation_source=""Manihotesculenta""","db_xref=""taxon:454143""","segment=""DNAA-like""","country=""India:Kerala",Pulikkal,"Malappuram""",NaN,NaN,NaN,NaN,NaN
4,5,source1..2757,"organism=""SriLankancassavamosaicvirus""","mol_type=""genomicDNA""","strain=""Erode:2011""","host=""Manihotesculenta""","db_xref=""taxon:161378""","segment=""DNA-A""","country=""India:Erode""","altitude=""183m""","collection_date=""29-Sep-2011""","collected_by=""AkhileshKumarKushawahaandIndran...",NaN,NaN,NaN,NaN


In [16]:
blast[4].str.split('=')

0         [ isolate, "SLCMV-Col"]
1            [ isolate, "Attur2"]
2       [ isolate, "DSMZPV-0424"]
3          [ isolate, "KeralaC4"]
4         [ strain, "Erode:2011"]
                 ...             
94              [ isolate, "CR2"]
95     [ isolate, "Katarniaghat"]
96              [ isolate, "CR1"]
97    [ isolate, "Katerniaghat2"]
98              [ serovar, "EK-"]
Name: 4, Length: 99, dtype: object